In [1]:
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
X = pd.read_csv('train.csv', index_col = 'Id')
X_test = pd.read_csv('test.csv', index_col = 'Id')

In [3]:
X.dropna(axis=0,subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [4]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 10 and 
                    X[cname].dtype == "object"]

numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]


In [6]:
##Data Normailzation

mean = X[numerical_cols].mean(axis = 0)
X[numerical_cols] -= mean
std = X[numerical_cols].std(axis=0)
X[numerical_cols] /= std

X_test[numerical_cols] -= mean
X_test[numerical_cols] /= std

In [7]:
# Imputing section : imputing all missing values
# numbers : median value
# strings : most frequent

numeric_processor = SimpleImputer(strategy='median')
categorical_processor = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_processor, numerical_cols),
        ('cat', categorical_processor, categorical_cols)
    ]
)

X = pd.DataFrame(preprocessor.fit_transform(X))
X.head()

,0,1,2,3,4,5,6,7,8,9,...,222,223,224,225,226,227,228,229,230,231
0,0.073350,-0.207948,-0.207071,0.651256,-0.517023,1.050634,0.878367,0.509840,0.575228,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.872264,0.409724,-0.091855,-0.071812,2.178881,0.156680,-0.429430,-0.572637,1.171591,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.073350,-0.084413,0.073455,0.651256,-0.517023,0.984415,0.829930,0.322063,0.092875,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.309753,-0.413838,-0.096864,0.651256,-0.517023,-1.862993,-0.720051,-0.572637,-0.499103,-0.288554,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.073350,0.574436,0.375020,1.374324,-0.517023,0.951306,0.733056,1.360357,0.463410,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
temp_id = X_test.index
print(temp_id)
X_test = pd.DataFrame(preprocessor.transform(X_test))
X_test.head()

Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name='Id', length=1459)


,0,1,2,3,4,5,6,7,8,9,...,222,223,224,225,226,227,228,229,230,231
0,-0.872264,0.409724,0.110725,-0.794879,0.381612,-0.339961,-1.155984,-0.572637,0.053410,0.604086,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.872264,0.450902,0.375721,-0.071812,0.381612,-0.439289,-1.301294,0.023830,1.051003,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.073350,0.162655,0.331939,-0.794879,-0.517023,0.851977,0.636183,-0.572637,0.761591,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.073350,0.327368,-0.053984,-0.071812,0.381612,0.885087,0.636183,-0.462180,0.347207,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.491770,-1.113866,-0.552217,1.374324,-0.517023,0.686430,0.345561,-0.572637,-0.396055,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [10]:
def build_network(input_shape = (0,)) :
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = input_shape))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'adam', loss = 'mae', metrics=['accuracy'])
    return model

In [11]:
model = build_network((X_train.shape[1],))
history = model.fit(X_train, y_train, epochs = 20,
                    validation_data = (X_valid, y_valid),
                   batch_size = 1)

Instructions for updating:
Colocations handled automatically by placer.
Train on 1168 samples, validate on 292 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1168/1168 [==============================] - 4s 3ms/sample - loss: 107884.5308 - acc: 0.0000e+00 - val_loss: 40830.3118 - val_acc: 0.0000e+00
Epoch 2/20
1168/1168 [==============================] - 3s 2ms/sample - loss: 31908.7293 - acc: 0.0000e+00 - val_loss: 24708.0902 - val_acc: 0.0000e+00
Epoch 3/20
1168/1168 [==============================] - 3s 2ms/sample - loss: 24017.0614 - acc: 0.0000e+00 - val_loss: 22984.8241 - val_acc: 0.0000e+00
Epoch 4/20
1168/1168 [==============================] - 5s 4ms/sample - loss: 21902.0457 - acc: 0.0000e+00 - val_loss: 19709.7549 - val_acc: 0.0000e+00
Epoch 5/20
1168/1168 [==============================] - 3s 3ms/sample - loss: 20347.7104 - acc: 0.0000e+00 - val_loss: 18436.4246 - val_acc: 0.0000e+00
Epoch 6/20
1168/1168 [==============================] - 3s 2ms/sample - 

In [12]:
pred = model.predict(X_valid)

In [13]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_valid,pred))

14145.10324807363


In [14]:
model = build_network((X.shape[1],))
history = model.fit(X, y, epochs = 20,
                   batch_size = 1)

Epoch 1/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 92975.0637 - acc: 0.0000e+00
Epoch 2/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 27842.3486 - acc: 0.0000e+00
Epoch 3/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 22491.2462 - acc: 0.0000e+00
Epoch 4/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 20640.5539 - acc: 0.0000e+00
Epoch 5/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 19427.5436 - acc: 0.0000e+00
Epoch 6/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 18466.5127 - acc: 0.0000e+00
Epoch 7/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 17611.2472 - acc: 0.0000e+00
Epoch 8/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 17088.5545 - acc: 0.0000e+00
Epoch 9/20
1460/1460 [==============================] - 3s 2ms/sample - loss: 16654.0199 - acc: 0.0000e+00
Epoch 10/20
1460/1460 [==============

In [15]:
preds_test = model.predict(X_test)
output = pd.DataFrame({'Id': temp_id,
                       'SalePrice': preds_test.flatten()})
X_test.head()


,0,1,2,3,4,5,6,7,8,9,...,222,223,224,225,226,227,228,229,230,231
0,-0.872264,0.409724,0.110725,-0.794879,0.381612,-0.339961,-1.155984,-0.572637,0.053410,0.604086,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.872264,0.450902,0.375721,-0.071812,0.381612,-0.439289,-1.301294,0.023830,1.051003,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.073350,0.162655,0.331939,-0.794879,-0.517023,0.851977,0.636183,-0.572637,0.761591,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.073350,0.327368,-0.053984,-0.071812,0.381612,0.885087,0.636183,-0.462180,0.347207,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.491770,-1.113866,-0.552217,1.374324,-0.517023,0.686430,0.345561,-0.572637,-0.396055,-0.288554,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
output.to_csv('submission.csv', index=False)